In [1]:
!pip install import_ipynb
!pip install keras-unet-collection
!pip install tensorflow==2.14.1

In [2]:
import import_ipynb
%run /content/drive/MyDrive/ColabNotebooks/Sementatic_Segmentation/UpdatedMeanIoU.ipynb
from keras_unet_collection import models

In [3]:
#from unet import build_unet
import os
import numpy as np
from matplotlib import pyplot as plt
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.metrics import MeanIoU
from keras.callbacks import EarlyStopping
import tensorflow as tf

In [4]:
from keras_unet_collection.layer_utils import *
from keras_unet_collection.activations import *

In [5]:
#New generator with rotation and shear where interpolation that comes with rotation and shear are thresholded in masks.
#This gives a binary mask rather than a mask with interpolated values.
seed=24
batch_size= 8
n_classes= 9

In [6]:
img_data_gen_args = dict(rescale = 1/255., #Original pixel values are 0 and 255. So rescaling to 0 to 1
                      width_shift_range=0.3,
                      height_shift_range=0.3,
                      shear_range=0.5,
                      horizontal_flip=True,
                      vertical_flip=True,
                      fill_mode='reflect')

In [7]:
#If You need to resize images then add this to the flow_from_directory parameters
#target_size=(150, 150), #Or whatever the size is for your network

image_data_generator = ImageDataGenerator(**img_data_gen_args)
image_generator = image_data_generator.flow_from_directory("/content/drive/MyDrive/ColabNotebooks/Sementatic_Segmentation/dataset_small/train_frame/",
                                                           seed=seed,
                                                           batch_size=batch_size,
                                                           class_mode=None)  #Very important to set this otherwise it returns multiple numpy arrays
                                                                            #thinking class mode is binary.

mask_generator = image_data_generator.flow_from_directory("/content/drive/MyDrive/ColabNotebooks/Sementatic_Segmentation/dataset_small/train_mask/",
                                                         seed=seed,
                                                         batch_size=batch_size,
                                                         color_mode = 'grayscale',   #Read masks in grayscale
                                                         class_mode=None)


valid_img_generator = image_data_generator.flow_from_directory("/content/drive/MyDrive/ColabNotebooks/Sementatic_Segmentation/dataset_small/val_frame/",
                                                               seed=seed,
                                                               batch_size=batch_size,
                                                               class_mode=None) #Default batch size 32, if not specified here
valid_mask_generator = image_data_generator.flow_from_directory("/content/drive/MyDrive/ColabNotebooks/Sementatic_Segmentation/dataset_small/val_mask/",
                                                               seed=seed,
                                                               batch_size=batch_size,
                                                               color_mode = 'grayscale',   #Read masks in grayscale
                                                               class_mode=None)  #Default batch size 32, if not specified here




Found 200 images belonging to 1 classes.
Found 200 images belonging to 1 classes.
Found 20 images belonging to 1 classes.
Found 20 images belonging to 1 classes.


In [8]:
test_img_generator = image_data_generator.flow_from_directory("/content/drive/MyDrive/ColabNotebooks/Sementatic_Segmentation/dataset_small/test_frame/",
                                                              seed=seed,
                                                              batch_size=32,
                                                              class_mode=None) #Default batch size 32, if not specified here

test_mask_generator = image_data_generator.flow_from_directory("/content/drive/MyDrive/ColabNotebooks/Sementatic_Segmentation/dataset_small/test_mask/",
                                                              seed=seed,
                                                              batch_size=32,
                                                              color_mode = 'grayscale',   #Read masks in grayscale
                                                              class_mode=None)  #Default batch size 32, if not specified here


Found 20 images belonging to 1 classes.
Found 20 images belonging to 1 classes.


In [9]:
from keras import backend as K

#Dice metric can be a great metric to track accuracy of semantic segmentation.
def dice_metric(y_pred, y_true):
    intersection = K.sum(K.sum(K.abs(y_true * y_pred), axis=-1))
    union = K.sum(K.sum(K.abs(y_true) + K.abs(y_pred), axis=-1))
    # if y_pred.sum() == 0 and y_pred.sum() == 0:
    #     return 1.0

    return 2*intersection / union


In [10]:
train_generator = zip(image_generator,mask_generator)
val_generator = zip(valid_img_generator, valid_mask_generator)
test_generator = zip(test_img_generator, test_mask_generator)

In [11]:
model_Unet = models.unet_2d((256, 256, 3), filter_num=[64, 128, 256, 512, 1024], n_labels=9, stack_num_down=2, stack_num_up=2,
                           activation='ReLU', output_activation='Softmax', batch_norm=True, pool=False, unpool=False,
                           backbone='VGG16', weights='imagenet', freeze_backbone=True, freeze_batch_norm=True, name='unet')

mIOU = UpdatedMeanIoU(num_classes=9)
model_Unet.compile(optimizer=Adam(lr = 1e-3), loss='categorical_crossentropy', metrics=[mIOU, dice_metric])
print(model_Unet.summary())

early_stopping = EarlyStopping(monitor="val_loss", patience=20, verbose=1, restore_best_weights=True,)


/usr/local/lib/python3.10/dist-packages/keras_unet_collection/_backbone_zoo.py:45: UserWarning: 

Backbone VGG16 does not use batch norm, but other layers received batch_norm=True
  warnings.warn(param_mismatch);


Model: "unet_model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 256, 256, 3)]        0         []                            
                                                                                                  
 VGG16_backbone (Functional  [(None, 256, 256, 64),       1471468   ['input_1[0][0]']             
 )                            (None, 128, 128, 128),      8                                       
                              (None, 64, 64, 256),                                                
                              (None, 32, 32, 512),                                                
                              (None, 16, 16, 512)]                                                
                                                                                         

In [ ]:
num_train_imgs = len(os.listdir('/content/drive/MyDrive/ColabNotebooks/Sementatic_Segmentation/dataset_small/train_frame/frame/'))
num_train_mask = len(os.listdir('/content/drive/MyDrive/ColabNotebooks/Sementatic_Segmentation/dataset_small/train_mask/mask/'))
print(num_train_imgs, num_train_mask)
steps_per_epoch = num_train_imgs //batch_size
print(steps_per_epoch)

history1=model_Unet.fit(train_generator, validation_data=val_generator, steps_per_epoch=steps_per_epoch,
                  validation_steps=steps_per_epoch, epochs=1,  callbacks=[early_stopping],)

200 200
25
23/25 [==========================>...] - ETA: 1:10 - loss: 0.0558 - updated_mean_io_u: 0.0574 - dice_metric: 0.0052

In [ ]:
print(test_img_generator[0].shape)
num_test_imgs = len(test_img_generator[0])
print(num_test_imgs)
batch_size = 16

steps_per_epoch = num_test_imgs //batch_size
print(steps_per_epoch)

_, dice_metric, mIOU = model_Unet.evaluate(test_generator, steps=steps_per_epoch)
print("Dice metric is = ", (dice_metric * 100.0), "%", "mIOU is = ", (mIOU * 100.0), "%")

In [ ]:
model_swin_unet_2d = models.resunet_a_2d((128, 128, 3), [32, 64, 128, 256, 512, 1024],
                            dilation_num=[[1, 3, 15, 31], [1, 3, 15, 31], [1, 3, 15], [1, 3, 15], [1,], [1,],],
                            n_labels=16, aspp_num_down=256, aspp_num_up=128,
                            activation='ReLU', output_activation='Sigmoid',
                            batch_norm=True, pool=False, unpool='nearest', name='resunet')

model_swin_unet_2d.compile(optimizer=Adam(lr = 1e-3), loss='categorical_crossentropy', metrics=[mIOU, dice_metric])
print(model_swin_unet_2d.summary())

early_stopping = EarlyStopping(monitor="val_loss", patience=20, verbose=1, restore_best_weights=True,)

In [ ]:
history2=model_swin_unet_2d.fit(train_generator, validation_data=val_generator, steps_per_epoch=steps_per_epoch,
                  validation_steps=steps_per_epoch, epochs=1,  callbacks=[early_stopping],)

In [ ]:
_, dice_metric, mIOU = model_Unet.evaluate(test_generator, steps=steps_per_epoch)
print("Dice metric is = ", (dice_metric * 100.0), "%", "mIOU is = ", (mIOU * 100.0), "%")

In [ ]:
#model.save('semanticSegmentation_50epochs.hdf5')
#model = tf.keras.models.load_model("semanticSegmentation_50epochs.hdf5", compile=False)